In [1]:
!pip install ipywidgets
!pip install datasets
!pip install huggingface_hub
!pip install pyarrow==15.0.2

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 474.3/474.3 kB 3.4 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 39.9/39.9 MB 13.1 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 134.8/134.8 kB 5.1 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 194.1/194.1 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2


In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import time
import json
import os
from datasets import load_dataset

In [4]:
url = 'https://www.house.kg/kupit-kvartiru'
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

In [6]:
pages = soup.find_all("li", class_="page-item")[-1].find("a", class_="page-link").get("data-page")
pages

'651'

In [9]:
with open(f'house_kg.json', 'a', encoding="utf-8") as f:
  for page in tqdm(range(1, int(pages)+1), leave=True):
    url = f'https://www.house.kg/kupit-kvartiru?page={page}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    objects = soup.find("div", class_="listings-wrapper")
    lincs = objects.find_all('div', class_="left-image")
    for ur in tqdm(lincs, leave=False):
      det_url = "https://www.house.kg" + ur.find('a')['href']
      response = requests.get(det_url)
      soup = BeautifulSoup(response.text, "html.parser")
      atr_dict = {}
      atr_dict['url'] = det_url

      atr_dict['usd_price'] = soup.find("div", class_="price-dollar").text.strip().replace(' ', '').replace('$', '')
      atr_dict['kgs_price'] = soup.find("div", class_="price-som").text.strip().replace(' ', '').replace('—Å–æ–º', '')
      atr_dict['–ö–æ–º–Ω–∞—Ç—ã'] = soup.find('div', class_='left').find('h1').text.strip()[0]
      atr_dict['–ê–¥—Ä–µ—Å'] = soup.find('div', class_='left').find('div', class_='address').text.strip()
      atr_dict['–¢–µ–ª–µ—Ñ–æ–Ω_–ø—Ä–æ–¥–∞–≤—Ü–∞'] = soup.find("div", class_="number").text
      atr_dict['–î–æ–±–∞–≤–ª–µ–Ω–æ'] = soup.find('span', class_ = 'added-span').text
      atr_dict['–ü—Ä–æ—Å–º–æ—Ç—Ä—ã'] = soup.find('span', class_ = 'view-count').text.strip()
      try:
        atr_dict['–ü–æ–¥–Ω—è—Ç–æ'] = soup.find('span', class_ = 'upped-span').text.strip()
      except Exception:
        pass
      try:
        atr_dict['–û–ø–∏—Å–∞–Ω–∏–µ'] = soup.find("div", class_="description").find('p').text
      except Exception:
        pass
      try:
        atr_dict['–†–µ–π—Ç–∏–Ω–≥'] = soup.find('div', class_ = 'rating score').text.strip()
      except Exception:
        pass
      try:
        atr_dict['–õ–∞–π–∫–∏'] = soup.find('span', class_ = 'favourite-count table-comments').text.strip()
      except Exception:
        pass
      atr_dict['–î–æ–ª–≥–æ—Ç–∞'] = soup.find("div", class_="map-wrapper").find("div", id="map2gis").get("data-lon")
      atr_dict['–®–∏—Ä–æ—Ç–∞'] = soup.find("div", class_="map-wrapper").find("div", id="map2gis").get("data-lat")
      atributs = soup.find_all("div", class_="info-row")
      for atr in atributs:
        key = atr.find('div', class_="label").text.strip().replace('\n', ' ')
        val = atr.find('div', class_="info").text.strip().replace('\n', ' ')
        atr_dict[key] = val

      json.dump(atr_dict, f, ensure_ascii=False)
      f.write('\n')

  0%|          | 0/651 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [20]:
for page in tqdm(range(1, int(pages)+1), leave=True):
    url = f'https://www.house.kg/kupit-kvartiru?page={page}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    objects = soup.find("div", class_="listings-wrapper")
    lincs = objects.find_all('div', class_="left-image")
    for ur in tqdm(lincs, leave=False):
      det_url = "https://www.house.kg" + ur.find('a')['href']
      response = requests.get(det_url)
      soup = BeautifulSoup(response.text, "html.parser")
      images = soup.find("div", class_="fotorama").find_all('a')[:-1]
      for img in tqdm(images, leave=False):
        image = requests.get(img['href']).content
        filename = f'images/{det_url.split("/")[-1]}/{img["href"].split("/")[-1]}'
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename, 'wb') as f:
          f.write(image)

  0%|          | 0/651 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [10]:
df = pd.read_json('house_kg.json', lines=True)

In [11]:
df.to_csv('house_kg.csv', index=False)

In [12]:
df.shape

(3010, 41)

In [13]:
df.head()

url  usd_price  kgs_price  \
0  https://www.house.kg/details/932931666cc8a829e...      97000    8184860   
1  https://www.house.kg/details/521743066ec024c57...     220000   18563600   
2  https://www.house.kg/details/141579566ed52b0b8...      85000    7172300   
3  https://www.house.kg/details/757010366ed4f9839...      85500    7214490   
4  https://www.house.kg/details/957794666ebe80157...     100000    8438000   

   –ö–æ–º–Ω–∞—Ç—ã                                            –ê–¥—Ä–µ—Å  \
0        3       –ë–∏—à–∫–µ–∫, –ú–∞–≥–∏—Å—Ç—Ä–∞–ª—å, –ú–∞–≥–∏—Å—Ç—Ä–∞–ª—å/–°—É—Ö—ç –ë–∞—Ç–æ—Ä–∞   
1        4  –ë–∏—à–∫–µ–∫, –ü–∞—Ä–∫ –ü–∞–Ω—Ñ–∏–ª–æ–≤–∞/–°–ø–∞—Ä—Ç–∞–∫, –ü–∞–Ω—Ñ–∏–ª–æ–≤–∞ 190/1   
2        1    –ë–∏—à–∫–µ–∫, –ú–∞–≥–∏—Å—Ç—Ä–∞–ª—å, –°—É—Ö—ç-–ë–∞—Ç–æ—Ä–∞ 24/–¢–æ–∫–æ–º–±–∞–µ–≤–∞   
3        2    –ë–∏—à–∫–µ–∫, –ú–∞–≥–∏—Å—Ç—Ä–∞–ª—å, –°—É—Ö—ç-–ë–∞—Ç–æ—Ä–∞ 24/–¢–æ–∫–æ–º–±–∞–µ–≤–∞   
4        4   –ë–∏—à–∫–µ–∫, –°—Ä–µ–¥–Ω–∏–π –î–∂–∞–ª –º-–Ω, –î–∂–∞–ª 23 16/–¢—ã–Ω–∞–ª–∏–µ–≤–∞   

    –¢–µ–ª–µ—Ñ–æ–Ω_–ø—Ä–æ–¥–∞–≤—Ü–∞                  –î–æ–±–∞–≤–ª–µ–Ω–æ  –ü—Ä–æ—Å–º–æ—Ç—Ä—ã  \
0  +996 500 07-57-77    –î–æ–±–∞–≤–ª–µ–Ω–æ 25 –¥–Ω–µ–π –Ω–∞–∑–∞–¥       4376   
1  +996 557 50-55-75     –î–æ–±–∞–≤–ª–µ–Ω–æ 1 –¥–µ–Ω—å –Ω–∞–∑–∞–¥       1118   
2  +996 706 89-49-26  –î–æ–±–∞–≤–ª–µ–Ω–æ 34 –º–∏–Ω—É—Ç—ã –Ω–∞–∑–∞–¥         41   
3  +996 995 56-55-56   –î–æ–±–∞–≤–ª–µ–Ω–æ 48 –º–∏–Ω—É—Ç –Ω–∞–∑–∞–¥         33   
4  +996 500 55-54-54     –î–æ–±–∞–≤–ª–µ–Ω–æ 1 –¥–µ–Ω—å –Ω–∞–∑–∞–¥        579   

                  –ü–æ–¥–Ω—è—Ç–æ                                           –û–ø–∏—Å–∞–Ω–∏–µ  \
0     –ü–æ–¥–Ω—è—Ç–æ 1 —á–∞—Å –Ω–∞–∑–∞–¥  –í —Å—Ä–æ—á–Ω–æ–π –ø—Ä–æ–¥–∞–∂–µ 3-—Ö –∫–æ–º–Ω–∞—Ç–Ω–∞—è –∫–≤–∞—Ä—Ç–∏—Ä–∞ —Å—Ç—É–¥–∏...   
1   –ü–æ–¥–Ω—è—Ç–æ 7 —á–∞—Å–æ–≤ –Ω–∞–∑–∞–¥  –ü—Ä–æ–¥–∞–µ—Ç—Å—è 4 –∫–æ–º–Ω–∞—Ç–Ω–∞—è –∫–≤–∞—Ä—Ç–∏—Ä–∞ –ø–æ —Å—Ä–æ—á–Ω–æ–π —Ü–µ–Ω–µ...   
2  –ü–æ–¥–Ω—è—Ç–æ 26 –º–∏–Ω—É—Ç –Ω–∞–∑–∞–¥  –ü—Ä–æ–¥–∞—ë—Ç—Å—è 2 –∫–æ–º–Ω–∞—Ç–Ω–∞—è —Å—Ç—É–¥–∏–π–Ω–∞—è –∫–≤–∞—Ä—Ç–∏—Ä–∞ –≤ —Ä–∞–π...   
3  –ü–æ–¥–Ω—è—Ç–æ 26 –º–∏–Ω—É—Ç –Ω–∞–∑–∞–¥  –ü—Ä–æ–¥–∞—ë—Ç—Å—è 2 –∫–æ–º–Ω–∞—Ç–Ω–∞—è —Å—Ç—É–¥–∏–π–Ω–∞—è –∫–≤–∞—Ä—Ç–∏—Ä–∞ –≤ —Ä–∞–π...   
4     –ü–æ–¥–Ω—è—Ç–æ 1 —á–∞—Å –Ω–∞–∑–∞–¥  ‚ÄºÔ∏è–°—Ä–æ—á–Ω–æ ‚ÄºÔ∏è\r\nüè¢ –ü—Ä–æ–¥–∞—é 4 - –∫–æ–º–Ω–∞—Ç—É—é –∫–≤–∞—Ä—Ç–∏—Ä—É ...   

   ...                 –ú–µ–±–µ–ª—å     –ü–æ–ª  \
0  ...                    NaN     NaN   
1  ...                    NaN     NaN   
2  ...  –ø–æ–ª–Ω–æ—Å—Ç—å—é –º–µ–±–ª–∏—Ä–æ–≤–∞–Ω–∞  –ø–∞—Ä–∫–µ—Ç   
3  ...  –ø–æ–ª–Ω–æ—Å—Ç—å—é –º–µ–±–ª–∏—Ä–æ–≤–∞–Ω–∞  –ø–∞—Ä–∫–µ—Ç   
4  ...   —á–∞—Å—Ç–∏—á–Ω–æ –º–µ–±–ª–∏—Ä–æ–≤–∞–Ω–∞     NaN   

                                        –ë–µ–∑–æ–ø–∞—Å–Ω–æ—Å—Ç—å –í–æ–∑–º–æ–∂–Ω–æ—Å—Ç—å —Ä–∞—Å—Å—Ä–æ—á–∫–∏  \
0                                                NaN                   NaN   
1                                                NaN                   NaN   
2  —Å–∏–≥–Ω–∞–ª–∏–∑–∞—Ü–∏—è,                                 ...                   –Ω–µ—Ç   
3  –æ—Ö—Ä–∞–Ω–∞,                                       ...                   NaN   
4  —Ä–µ—à–µ—Ç–∫–∏ –Ω–∞ –æ–∫–Ω–∞—Ö,                             ...                   NaN   

  –í–æ–∑–º–æ–∂–Ω–æ—Å—Ç—å –∏–ø–æ—Ç–µ–∫–∏   –í–æ–∑–º–æ–∂–Ω–æ—Å—Ç—å –æ–±–º–µ–Ω–∞ –ü–ª–æ—â–∞–¥—å —É—á–∞—Å—Ç–∫–∞ –ö–∞–Ω–∞–ª–∏–∑–∞—Ü–∏—è  \
0                 NaN                  NaN             NaN         NaN   
1                 NaN                  NaN             NaN         NaN   
2                 –Ω–µ—Ç                  NaN             NaN         NaN   
3                 NaN  –æ–±–º–µ–Ω –Ω–µ –ø—Ä–µ–¥–ª–∞–≥–∞—Ç—å             NaN         NaN   
4                –µ—Å—Ç—å                  NaN             NaN         NaN   

  –ü–∏—Ç—å–µ–≤–∞—è –≤–æ–¥–∞ –≠–ª–µ–∫—Ç—Ä–∏—á–µ—Å—Ç–≤–æ  
0           NaN           NaN  
1           NaN           NaN  
2           NaN           NaN  
3           NaN           NaN  
4           NaN           NaN  

[5 rows x 41 columns]

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3010 entries, 0 to 3009
Data columns (total 41 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   url                             3010 non-null   object 
 1   usd_price                       3010 non-null   int64  
 2   kgs_price                       3010 non-null   int64  
 3   –ö–æ–º–Ω–∞—Ç—ã                         3010 non-null   int64  
 4   –ê–¥—Ä–µ—Å                           3010 non-null   object 
 5   –¢–µ–ª–µ—Ñ–æ–Ω_–ø—Ä–æ–¥–∞–≤—Ü–∞                3010 non-null   object 
 6   –î–æ–±–∞–≤–ª–µ–Ω–æ                       3010 non-null   object 
 7   –ü—Ä–æ—Å–º–æ—Ç—Ä—ã                       3010 non-null   int64  
 8   –ü–æ–¥–Ω—è—Ç–æ                         2903 non-null   object 
 9   –û–ø–∏—Å–∞–Ω–∏–µ                        2936 non-null   object 
 10  –õ–∞–π–∫–∏                           2013 non-null   float64
 11  –î–æ–ª–≥–æ—Ç–∞                

In [18]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [26]:
dataset = load_dataset("imagefolder", data_dir="./images")

Resolving data files:   0%|          | 0/242 [00:00<?, ?it/s]

In [27]:
dataset.push_to_hub("sharshenaliev/images")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/239 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/sharshenaliev/images/commit/75b0e4c75b6cd057cc65b9adca4365fb40dc6304', commit_message='Upload dataset', commit_description='', oid='75b0e4c75b6cd057cc65b9adca4365fb40dc6304', pr_url=None, pr_revision=None, pr_num=None)

In [24]:
dataset = load_dataset("csv", data_files="house_kg.csv")

In [25]:
dataset.push_to_hub("sharshenaliev/house_kg")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/sharshenaliev/house_kg/commit/c5f33b5e582fa9f06509ae366b9ea6d9c5dd5466', commit_message='Upload dataset', commit_description='', oid='c5f33b5e582fa9f06509ae366b9ea6d9c5dd5466', pr_url=None, pr_revision=None, pr_num=None)